In [131]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import os

In [125]:
# 手动下好了MNIST训练集放在/raw中,然后使用了torch官方源文件中处理MNIST的方法,转化成torch能直接使用的数据了
from torchvision.datasets.mnist import read_image_file,read_label_file
training_set = (
            read_image_file(os.path.join('./MNIST/raw/', 'train-images-idx3-ubyte')),
            read_label_file(os.path.join('./MNIST/raw/', 'train-labels-idx1-ubyte'))
        )
test_set = (
            read_image_file(os.path.join('./MNIST/raw/', 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join('./MNIST/raw/', 't10k-labels-idx1-ubyte'))
        )
with open('./MNIST/processed/training.pt', 'wb') as f:
            torch.save(training_set, f)
with open('./MNIST/processed/test.pt', 'wb') as f:
            torch.save(test_set, f)

print('Done!')

Done!


In [132]:
# to tensor change data to (0,1),Normalize change it to (data-mean)/std 
# so  it's change from (0,1) to (-1,1)
trans = transforms.Compose([transforms.ToTensor,
                                transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))])
trainset = torchvision.datasets.MNIST(root='./MNIST',download=False,train=True,transform=trans)
train_loader = DataLoader(trainset,batch_size=16,shuffle=True,num_workers=2)
testset = torchvision.datasets.MNIST(root='./MNIST',download=False,train=False,transform=trans)
test_loader = DataLoader(trainset,batch_size=16,shuffle=True,num_workers=2)

In [96]:
#input image size : N,28,28,1
dtype = torch.FloatTensor
class Flatten(nn.Module):
    def forward(self,x):
        N = x.size()[0]
        return x.view(N,-1)

Net = nn.Sequential(
        nn.Conv2d(1,16,kernel_size=5),
        nn.BatchNorm2d(16),
        nn.Conv2d(16,32,kernel_size=5,stride=2),
        nn.BatchNorm2d(32),
        Flatten(),
        nn.Linear(32*10*10,1024),
        nn.ReLU(),
        nn.Linear(1024,10)
)
Net.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(Net.parameters(),lr=1e-2)

In [133]:
print(Net)
params = list(Net.parameters())

Sequential(
  (0): Conv2d (1, 16, kernel_size=(5, 5), stride=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (2): Conv2d (16, 32, kernel_size=(5, 5), stride=(2, 2))
  (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (4): Flatten(
  )
  (5): Linear(in_features=4608, out_features=1024)
  (6): ReLU()
  (7): Linear(in_features=1024, out_features=10)
)


In [ ]:
# 设置好了数据集,也创建了一个网络,下一步就是使用数据来训练网络了